<a href="https://colab.research.google.com/github/O-Kpy/Project/blob/main/%EB%89%B4%EC%8A%A4%EA%B8%B0%EC%82%AC_%ED%81%AC%EB%A1%A4%EB%9F%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import pandas as pd
import re

'''''''''''''''''''''''''''''''''''''''''''''''''''''''''
< naver 뉴스 검색시 리스트 크롤링하는 프로그램 > _select사용
- 크롤링 해오는 것 : 링크,제목,신문사,내용요약본
- 내용요약본  -> 정제 작업 필요
- 리스트 -> 딕셔너리 -> df -> csv로 저장
'''''''''''''''''''''

# 각 크롤링 결과 저장하기 위한 리스트 선언
title_text=[]
link_text=[]
source_text=[]
contents_text=[]
result={}

RESULT_PATH ='/content/'  # 결과 저장할 경로

# 내용 전처리
def contents_cleansing(contents):
    first_cleansing_contents = re.sub('<dl>.*?</a> </div> </dd> <dd>', '',str(contents)).strip()  # 앞에 필요없는 부분 제거
    second_cleansing_contents = re.sub('<ul class="relation_lst">.*?</dd>', '', first_cleansing_contents).strip()  # 뒤에 필요없는 부분 제거

 
    third_cleansing_contents = re.sub('<.+?>', '', second_cleansing_contents).strip()
    contents_text.append(third_cleansing_contents)
    #print(contents_text)

# 크롤링 시작
def crawler(maxpage,query,sort,s_date,e_date):
    s_from = s_date.replace(".","")
    e_to = e_date.replace(".","")
    page = 1
    maxpage_t =(int(maxpage)-1)*10+1
    while page <= maxpage_t:
        url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort="+sort+"&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
        response = requests.get(url)
        html = response.text

        # BeautifulSoup 인자값 지정
        soup = BeautifulSoup(html, 'html.parser')

        # <a>태그에서 제목과 링크주소 (a 태그 중 class 명이 news_tit인 것)
        atags = soup.find_all('a', 'news_tit')
        for atag in atags:
            title = atag.get('title')
            title_text.append(title)     # 제목
            link_text.append(atag['href'])   # 링크주소

        # 신문사 추출 (a 태그 중 class 명이 info press인 것)
        source_lists = soup.find_all('a', 'info press')
        for source_list in source_lists:
            source_text.append(source_list.text)    # 신문사

        # 본문요약본 (a 태그 중 class 명이 api_txt_lines dsc_txt_wrap인 것)
        contents_lists = soup.find_all('a','api_txt_lines dsc_txt_wrap')
        for contents_list in contents_lists:
            contents_cleansing(contents_list) # 본문요약 정제화

        # 모든 리스트 딕셔너리형태로 저장
        result= {"title":title_text ,  "source" : source_text ,"link":link_text, "subtitle":query }
        df = pd.DataFrame(result)  # df로 변환
        page += 10

    # 새로 만들 파일이름 지정
    outputFileName = f'{query} merging.csv'
    df.to_csv(RESULT_PATH+outputFileName, index=False)

def main():
    info_main = input("="*50+"\n"+"입력 형식에 맞게 입력해주세요."+"\n"+" 시작하시려면 Enter를 눌러주세요."+"\n"+"="*50)
    maxpage = input("최대 크롤링할 페이지 수 입력하시오: ")
    query = input("검색어 입력: ")
    sort = input("뉴스 검색 방식 입력(관련도순=0  최신순=1  오래된순=2): ")
    s_date = input("시작날짜 입력(2019.01.04):")
    e_date = input("끝 날짜 입력(2019.01.05):")
    crawler(maxpage,query,sort,s_date,e_date)

main()


입력 형식에 맞게 입력해주세요.
 시작하시려면 Enter를 눌러주세요.
최대 크롤링할 페이지 수 입력하시오: 2
검색어 입력: 청년친화강소기업
뉴스 검색 방식 입력(관련도순=0  최신순=1  오래된순=2): 0
시작날짜 입력(2019.01.04):2019.01.01
끝 날짜 입력(2019.01.05):2021.09.09


In [5]:
import pandas as pd
cr = pd.read_csv('/content/청년친화강소기업 merging.csv')
cr

,title,source,link,subtitle
0,원광대 WK300 강소기업 66.7％ 하반기 인력 채용 추진,뉴시스,http://www.newsis.com/view/?id=NISX20210909_00...,청년친화강소기업
1,"정부, 우수 중기-청년 10만명 직접 매칭",ZDNet Korea,https://zdnet.co.kr/view/?no=20210909101629,청년친화강소기업
2,"고용노동부, 청년친화강소기업 신청 접수",전자신문언론사 선정,http://www.etnews.com/20210826000083,청년친화강소기업
3,"연세대 미래캠퍼스, 지역청년 위한 온라인 청년고용정책 소개 프로그램 성료",프레시안,https://www.pressian.com/pages/articles/202109...,청년친화강소기업
4,"""공공기관 청년고용의무제 2023년까지 2년 연장""…정원 3% 의무 고용",파이낸셜뉴스언론사 선정,http://www.fnnews.com/news/202108171036029352,청년친화강소기업
5,2022 청년친화강소기업 신청 받아...9월 16일까지,아웃소싱타임스,http://www.outsourcing.co.kr/news/articleView....,청년친화강소기업
6,김지현의 청년 관점 <6> 노동 정책 혁신 방향,국제신문,http://www.kookje.co.kr/news2011/asp/newsbody....,청년친화강소기업
7,"[강소기업 돋보기] “기업의 꾸준한 성장과 비전, 젊은 인재 확보의 핵심” - ㈜DKM",산업일보,http://www.kidd.co.kr/news/223823,청년친화강소기업
8,공공기관 청년고용의무제 2023년까지 연장,매일노동뉴스,http://www.labortoday.co.kr/news/articleView.h...,청년친화강소기업
9,"野박진 ""법인세 한시 폐지, 규제프리존, 겸직완화"" 청년일자리 공약",디지털타임스,http://www.dt.co.kr/contents.html?article_no=2...,청년친화강소기업


In [ ]:
데이터 보고서 수정, 데이터 분석 그래프 및 설명글 작성 (기업 데이터 분석 카테고리)(그전에 대시보드 완성), 테이블 필요없는 컬럼날리기

In [ ]:
서비스 소개 > 개발과정에 넣을 본인의 개발 과정 글 작성 (사용한 프로그램, 구체적인 데이터 분석 과정)

In [ ]:
# aws Lambda를 사용하여 크롤링, 데이터 적재 해보기